for the next few notebooks im just going to mess around creating different features and seeing how they do against volatility scaled returns

in an effort for cleanliness, in this notebook i'm going to make a csv file that has the basics that can be imported so all notebook lengths can be reduced

In [22]:
# importing packages and data

import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np

df = pd.read_csv("extended_data.csv", 
dtype={0: float, 1: str, 2: float,  3: float,  4: float,  5: float,  6: float,  7: float,  8: int}
)

# fill missed funding periods with 0
df["funding"] = df["funding"].fillna(0)

# check number of assets
print(len(df["asset"].unique()))

#check if missing values remaining
nan_rows = df.isna().any(axis=1)
rows_with_nan = df[nan_rows]
print(rows_with_nan)

57
Empty DataFrame
Columns: [timestamp, asset, open, high, low, close, volume, funding, funding inaccurate]
Index: []


In [23]:
# define constants
RETURNS_LOOKBACK = 30
TARGET_VOLATILITY = 1/math.sqrt(365)
print(TARGET_VOLATILITY)

0.052342392259021375


In [24]:
# making time readable
df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")

# getting returns and return and funding volatility

# asset retuns
df["returns"] = (
    df.groupby("asset")["close"]
      .transform(lambda x: x.pct_change())
)

# asset returns inclusive of funding
df["returns_funding"] = df["returns"] - df["funding"]

# next day asset returns inclusive of funding
df["next_returns_funding"] = (
    df.groupby("asset")["returns_funding"]
      .shift(-1)
)

# asset returns and funding volatility
df["rolling_returns_funding_vol"] = (
    df.groupby("asset")["returns_funding"]
      .rolling(window=RETURNS_LOOKBACK)
      .std()
      .reset_index(level=0, drop=True)
)

# check work
df.iloc[50:56].head()

,timestamp,asset,open,high,low,close,volume,funding,funding inaccurate,returns,returns_funding,next_returns_funding,rolling_returns_funding_vol
50,2025-01-25 20:00:00,1000BONK-USD,0.029637,0.029813,0.028716,0.029260,574900.0,-0.000037,24,-0.012721,-0.012684,-0.154063,0.075372
51,2025-01-26 20:00:00,1000BONK-USD,0.029260,0.029260,0.024299,0.024748,1239300.0,-0.000141,24,-0.154204,-0.154063,0.008947,0.080323
52,2025-01-27 20:00:00,1000BONK-USD,0.024748,0.026785,0.024748,0.024954,982400.0,-0.000623,24,0.008324,0.008947,-0.001621,0.079880
53,2025-01-28 20:00:00,1000BONK-USD,0.024954,0.025741,0.022771,0.024913,1317900.0,-0.000022,24,-0.001643,-0.001621,0.014194,0.079750
54,2025-01-29 20:00:00,1000BONK-USD,0.024913,0.026438,0.023934,0.025254,1091300.0,-0.000506,24,0.013688,0.014194,-0.012058,0.079690


In [25]:
# creating funding sma

for i in [1, 3, 5, 10, 15, 25, 40, 55]:
    
    df[f"funding_sma{i}"] = (
    df.groupby("asset")["funding"]
      .rolling(window=i)
      .mean()
      .reset_index(level=0, drop=True)
    )

In [26]:
# scaling returns and funding by volatility
df["returns_funding_volscaled"] = df["returns_funding"]*(TARGET_VOLATILITY/df["rolling_returns_funding_vol"])

# next day asset returns and funding scaled by volatility
df["next_returns_funding_volscaled"] = (
    df.groupby("asset")["returns_funding_volscaled"]
      .shift(-1)
)

# editing to remove what wont yet include a volatility scale and final day
df_ = df[df.groupby('asset').cumcount() > 55].copy()
df_ = df_.groupby('asset').head(-1).copy()

#check if missing values remaining
nan_rows = df_.isna().any(axis=1)
rows_with_nan = df_[nan_rows]
print(rows_with_nan)

Empty DataFrame
Columns: [timestamp, asset, open, high, low, close, volume, funding, funding inaccurate, returns, returns_funding, next_returns_funding, rolling_returns_funding_vol, funding_sma1, funding_sma3, funding_sma5, funding_sma10, funding_sma15, funding_sma25, funding_sma40, funding_sma55, returns_funding_volscaled, next_returns_funding_volscaled]
Index: []

[0 rows x 23 columns]


In [27]:
idx_max = df_["next_returns_funding_volscaled"].idxmax()
print(df_.loc[idx_max])
idx_min = df_["next_returns_funding_volscaled"].idxmin()
print(df_.loc[idx_min])

timestamp                         2025-02-28 20:00:00
asset                                         ADA-USD
open                                            0.636
high                                           0.6517
low                                            0.6282
close                                          0.6424
volume                                         1560.0
funding                                     -0.000301
funding inaccurate                                 24
returns                                      0.010063
returns_funding                              0.010364
next_returns_funding                         0.620888
rolling_returns_funding_vol                  0.042673
funding_sma1                                -0.000301
funding_sma3                                 0.000526
funding_sma5                                 0.000498
funding_sma10                               -0.000479
funding_sma15                               -0.000783
funding_sma25               

In [28]:
# save to csv
df.to_csv("extended_features.csv", index = False)